In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Load the CSV files
candidates_df = pd.read_excel('/kaggle/input/word2vec-lightcast/df-dropna.xlsx')
titles_df = pd.read_csv('/kaggle/input/word2vec-lightcast/lightcast-titles.csv')

In [3]:
import re

def clean_text_columns(df, columns):
    clean_pattern = r'[!@#$%^&*()_+{}\[\-]:;<>,.?~\\/|"]'
    for col in columns:
        df[col] = df[col].apply(lambda x: re.sub(clean_pattern, ' ', str(x)))
        df[col] = df[col].str.replace(r'\s+', ' ', regex=True).str.strip()
        df[col] = df[col].str.lower()  # Convert to lowercase
    return df

# List of columns to clean
columns_to_clean = ['candidate_profiles','designation']

# Call the clean_text_columns function to clean the specified columns
candidates_df = clean_text_columns(candidates_df, columns_to_clean)
candidates_df.sample(10)

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,job_categories,designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max
3322,ACCEPTED,3.83,12.5,14.37,15,freelance developer|||full stack react developer,Developer,mern stack developer,1,3.0,5.0,11.0,14.0
6144,ACCEPTED,3.00,42.0,52.00,30,software developer - ii (android),Backend Developer,backend developer - ii/ iii,1,2.0,7.0,20.0,50.0
11566,ACCEPTED,3.42,3.5,5.00,15,motion graphics designer|||vfx roto and paint ...,Designer,graphic and motion designer,1,4.0,10.0,5.0,10.0
10669,ACCEPTED,8.00,14.0,1700000.00,15,assistant manager|||assistant manager corporat...,Business development,sales development manager - mumbai,1,4.0,7.0,15.0,20.0
9996,ACCEPTED,4.00,20.0,36.00,7,sde 2,Backend Developer,sde iii - backend,1,5.0,8.0,35.0,55.0
8709,ACCEPTED,3.00,5.0,6.50,30,key relationship manager,Inside Sales (Outbound),business development associate,1,0.5,5.0,5.0,7.0
1678,ACCEPTED,6.00,18.0,24.00,30,associate consultant|||marketing manager|||onl...,Digital marketing,growth hacker,1,2.0,6.0,15.0,25.0
13295,REJECTED,5.40,17.0,20.00,30,media coordinator,IT instructor,product designer,0,2.0,5.0,10.0,30.0
7341,ACCEPTED,2.75,9.5,14.00,30,java developer,Backend Developer,sde- ii backend,1,2.0,5.0,25.0,40.0
11526,ACCEPTED,0.00,0.0,2.50,0,fse,Sales,business development executive,1,0.0,1.0,2.0,3.0


In [4]:
titles_df.sample(5)

,ID,Title
11368,ET17115A8F58E3FE47,Circulation Desk Assistant
63022,ET7B9D96B4D3BCBA34,Software Developer/Programmer
50023,ET38CC830CA53DCD5B,Permit Runner
46388,ET491E75E5F38AA886,Night Supervisor/Licensed Practical Nurse Lice...
55871,ET475C9860904FB571,Recall Coordinator


In [5]:
titles_df['Title'] = titles_df['Title'].str.lower()

In [6]:
# Initialize TfidfVectorizer
tfidf = TfidfVectorizer(vocabulary=titles_df['Title'])

# Transform candidate profiles to TfIdf vectors
candidate_vectors = tfidf.fit_transform(candidates_df['candidate_profiles'])  
designation_vectors = tfidf.fit_transform(candidates_df['designation'])

In [7]:
# Calculate cosine similarity between candidates & titles
candidates_cos_sim = cosine_similarity(candidate_vectors, tfidf.transform(titles_df['Title']))  
designation_cos_sim = cosine_similarity(designation_vectors, tfidf.transform(titles_df['Title']))  

In [14]:
candidates_cos_sim, designation_cos_sim

(array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]),
 array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        ...,
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 1., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]]))

In [8]:
# Get index of most similar title for each candidate
c_idx = candidates_cos_sim.argmax(axis=1)
d_idx = designation_cos_sim.argmax(axis=1)

In [9]:
# Add matching title as a new column 
candidates_df['normalized_candidate_profiles'] = titles_df.loc[c_idx, 'Title'].values
candidates_df['normalized_jd_designation'] = titles_df.loc[d_idx, 'Title'].values

In [10]:
candidates_df.head()

,status,experience,present_ctc,expected_ctc,notice_period,candidate_profiles,job_categories,designation,Relevant,exp_min,exp_max,ctc_fixed_min,ctc_fixed_max,normalized_candidate_profiles,normalized_jd_designation
0,ACCEPTED,2.00,3.6,4.14,15,senior sales executive,"Inside Sales (Outbound),Sales",inside sales,1,0.5,3.0,3.0,5.0,account delivery executive,.net application architect
1,ACCEPTED,11.00,3.0,4.00,1,marketing executive,"Inside Sales (Outbound),Sales",inside sales,1,0.5,3.0,3.0,5.0,account delivery executive,.net application architect
2,ACCEPTED,2.50,5.0,5.75,15,android developer,Frontend Developer,react native developer,1,2.0,4.0,6.0,10.0,.net application developer,.net application developer
3,ACCEPTED,4.08,7.0,8.05,15,android mobile application developer|||android...,Frontend Developer,react native developer,1,2.0,4.0,6.0,10.0,.net application developer,.net application developer
4,ACCEPTED,3.50,4.0,8.00,1,devops engineer,Developer Operations,developer operations,1,1.0,5.0,6.0,10.0,.net full stack engineer,.net application developer


In [11]:
# Reorder columns
columns_reordered = ['status', 'experience', 'present_ctc', 'expected_ctc', 'notice_period',
                     'candidate_profiles', 'normalized_candidate_profiles', 'job_categories', 
                     'designation', 'normalized_jd_designation', 'Relevant', 'exp_min', 'exp_max', 'ctc_fixed_min', 
                     'ctc_fixed_max']

# Reindex the DataFrame with the reordered columns
candidates_df = candidates_df.reindex(columns=columns_reordered)

print(candidates_df.head())  # Displaying the reordered DataFrame

     status  experience  present_ctc  expected_ctc  notice_period  \
0  ACCEPTED        2.00          3.6          4.14             15   
1  ACCEPTED       11.00          3.0          4.00              1   
2  ACCEPTED        2.50          5.0          5.75             15   
3  ACCEPTED        4.08          7.0          8.05             15   
4  ACCEPTED        3.50          4.0          8.00              1   

                                  candidate_profiles  \
0                             senior sales executive   
1                                marketing executive   
2                                  android developer   
3  android mobile application developer|||android...   
4                                    devops engineer   

  normalized_candidate_profiles                 job_categories  \
0    account delivery executive  Inside Sales (Outbound),Sales   
1    account delivery executive  Inside Sales (Outbound),Sales   
2    .net application developer            Fronten

In [12]:
candidates_df.to_csv('norm-profiles.csv', index=False)